In [38]:
import keras, os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
import keras.preprocessing
import numpy as np

from keras.applications import vgg16
from keras.optimizers import adam_v2
from keras.losses import categorical_crossentropy

import matplotlib.pyplot as plt

In [39]:
# Get train and test data from folder.
TRAIN_DIRECTORY = "AFD/AFD（Animal Face Dataset)/face images/Saimiri sciureus/Saimiri sciureus"
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32  # This is keras default
NUM_CHANNELS = 3
target_size = (IMG_HEIGHT, IMG_WIDTH)

In [40]:
train_datagen = ImageDataGenerator(validation_split=0.2)
train_generator = train_datagen.flow_from_directory(directory=TRAIN_DIRECTORY, target_size=target_size,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(directory=TRAIN_DIRECTORY, target_size=target_size,
                                                         batch_size=BATCH_SIZE,
                                                         shuffle=True,
                                                         subset='validation')

# testDataGenerator = ImageDataGenerator()
# testData = testDataGenerator.flow_from_directory(directory="test", target_size=target_size)

Found 220 images belonging to 44 classes.
Found 32 images belonging to 44 classes.


In [41]:
# vgg16.preprocess_input(
# )

NUMBER_INDIVIDUALS = 44
NUMBER_CLASSES = NUMBER_INDIVIDUALS + 2  # One for false-positive (not a monkey), the other for previously unseen individuals

# See here for implementation https://keras.io/api/applications/vgg/#vgg16-function
model = vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS),
    pooling='avg',  # Global average pooling applied.
)

In [45]:
NUM_EPOCHS = 20

optimizer = adam_v2.Adam(learning_rate=0.001)
loss = categorical_crossentropy

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [46]:
hist = model.fit(
    x=train_generator,  # y is obtained from this x
    steps_per_epoch=train_generator.samples,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples,
    epochs=NUM_EPOCHS)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [ ]:
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "loss", "Validation Loss"])
plt.show()